In [1]:
import os
import gc
import re
import sys
import time
import json
import random
import datetime
import traceback
import pickle
import pandas as pd
import numpy as np

## 根据原始车源json数据,生成款型待预测文件

In [103]:
f = open("../tmp/train/car.source.dump.json", encoding='utf-8')
count = 0
columns = ['car_application','city','created_on','domain','mile','month','price','title','transfer_owner','year']
final = pd.DataFrame([],columns=columns)
final.to_csv('../tmp/train/wait_predict.csv', index=False)
for text_json in f.readlines():
    detail = json.loads(text_json)
    result = pd.DataFrame(detail,index=[0])
    result.to_csv('../tmp/train/wait_predict.csv', header=False, mode='a', index=False)

In [104]:
result = pd.read_csv('../tmp/train/wait_predict.csv')
result = result.loc[(result['car_application'] == '非营运')&(result['transfer_owner'] == 0), :]
result = result.loc[(result['title'].notnull())&(result['price'].notnull())&(result['year'].notnull())&(result['mile'].notnull()), :]
result = result.reset_index().rename(columns={'index':'id'})
result.to_csv('../tmp/train/wait_predict.csv', index=False)
result.head()

,id,car_application,city,created_on,domain,mile,month,price,title,transfer_owner,year
0,0,非营运,青岛,2018-07-03 18:47:54,guazi.com,7.63,9,13.0,别克GL8 2014款 2.4L 经典版,0,2014
1,1,非营运,绵阳,2018-09-12 15:20:07,guazi.com,1.72,5,3.2,北汽幻速H2 2016款 H2E 1.5L 经典型BJ415A,0,2017
2,2,非营运,深圳,2018-06-26 10:14:10,guazi.com,9.15,7,9.5,日产 天籁 2012款 2.0L XL智享版,0,2012
3,3,非营运,珠海,2018-09-27 11:34:54,guazi.com,1.74,3,4.0,长安商用 2015款 1.5L金欧诺标准型,0,2017
4,4,非营运,福州,2018-09-25 14:02:18,guazi.com,3.79,9,30.0,奥迪Q5 2015款 40 TFSI 技术型,0,2015


In [105]:
result.describe()

,id,mile,month,price,transfer_owner,year
count,1.172847e+06,1.172847e+06,1.172847e+06,1.172847e+06,1172847.0,1.172847e+06
mean,5.949336e+05,6.076538e+00,6.557573e+00,9.721117e+00,0.0,2.082653e+03
std,3.518494e+05,4.058691e+00,3.512714e+00,1.089356e+01,0.0,5.398284e+02
min,0.000000e+00,1.000000e-02,1.000000e+00,1.000000e-02,0.0,1.003000e+03
25%,2.932245e+05,3.010000e+00,4.000000e+00,4.000000e+00,0.0,2.012000e+03
50%,5.864520e+05,5.330000e+00,7.000000e+00,6.800000e+00,0.0,2.014000e+03
75%,8.796805e+05,8.330000e+00,1.000000e+01,1.140000e+01,0.0,2.016000e+03
max,1.264004e+06,9.110000e+01,1.200000e+01,6.250000e+02,0.0,6.481000e+03


## 算法预测结果

In [32]:
def process_price_bn(df):
    if df['price_bn'] == '暂无报价':
        return np.NAN
    temp = re.findall(r'万',df['price_bn'])
    if len(temp) > 1:
        text = df['price_bn'].split('~')[1]
    else:
        text = df['price_bn']
    text = text.replace('万','')
    return float(text)

def process_online_year(df):
    return int(re.findall(r'(\d+)款',df['detail_name'])[0])

car_detail_info = pd.read_csv('../tmp/train/car_detail_info.csv')
car_detail_info = car_detail_info.drop(['text'],axis=1)
car_detail_info['predict_brand_slug'] = car_detail_info['predict_brand_slug'].fillna(-1)
car_detail_info['predict_brand_slug'] = car_detail_info['predict_brand_slug'].astype(int)
car_detail_info['predict_model_slug'] = car_detail_info['predict_model_slug'].fillna(-1)
car_detail_info['predict_model_slug'] = car_detail_info['predict_model_slug'].astype(int)
car_detail_info['predict_detail_slug'] = car_detail_info['predict_detail_slug'].fillna(-1)
car_detail_info['predict_detail_slug'] = car_detail_info['predict_detail_slug'].astype(int)
car_detail_info = car_detail_info.rename(columns={'predict_brand_slug':'brand_slug','predict_model_slug':'model_slug','predict_detail_slug':'detail_slug'})

car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')
final = car_detail_info.merge(car_autohome_all.loc[:,['brand_slug','brand_name']].drop_duplicates(['brand_slug','brand_name']), how='left', on=['brand_slug'])
final = final.merge(car_autohome_all.loc[:,['model_slug','model_name']].drop_duplicates(['model_slug','model_name']), how='left', on=['model_slug'])
final = final.merge(car_autohome_all.loc[:,['detail_slug','detail_name','price_bn','body','energy']], how='left', on=['detail_slug'])
final = final.loc[(final['model_slug'] != -1)&(final['detail_slug'] != -1)&(final['brand_slug'] != -1)&(final['price_bn'].notnull()), :].reset_index(drop=True)
final['price_bn'] = final.apply(process_price_bn, axis=1)
final = final.loc[(final['price_bn'].notnull()), :]
final = final.drop(['car_application','created_on','transfer_owner','id'], axis=1).reset_index().rename(columns={'index':'id'})
final['online_year'] = final.apply(process_online_year,axis=1)
final.to_csv('/home/ml/ProgramProject/evaluation-predict/tmp/train/car_source_match.csv', index=False)

## 直接字符串匹配

In [3]:
def process_price_bn(df):
    if df['price_bn'] == '暂无报价':
        return np.NAN
    temp = re.findall(r'万',df['price_bn'])
    if len(temp) > 1:
        text = df['price_bn'].split('~')[1]
    else:
        text = df['price_bn']
    text = text.replace('万','')
    return float(text)

def delete_str_useless(df, column_name):
    """
    删除没用的字符
    """    
    text = df[column_name]
    text = text.replace('·', '')
    text = text.replace('・', '')
    text = text.replace('-', '')
    text = text.replace(' ', '')
    text = text.lower()
    return text

# 判断是否有重复final_text
wait_predict = pd.read_csv('../tmp/train/wait_predict.csv')
wait_predict['title'] = wait_predict.apply(delete_str_useless, args=('title',), axis=1)
train_final = pd.read_csv('../tmp/train/train_final.csv')
train_final = train_final.rename(columns={'final_text':'title'})
data = train_final.groupby(['title'])['brand_slug'].count().reset_index()
data.loc[(data['brand_slug'] > 1), :]

,title,brand_slug


In [4]:
# data = wait_predict.loc[(wait_predict['domain'].isin(['guazi.com'])), :]
data = wait_predict.copy()
print('瓜子车源数据量:', len(data))
final = data.merge(train_final, how='left', on=['title'])
final = final.sort_values(by=['title'])
match = final.loc[(final['online_year'].notnull())&(final['city'].notnull()), :].reset_index(drop=True)
print('瓜子车源数据匹配量:',len(match), '匹配率:', len(match)/len(data))
match.info()
match.to_csv('/home/ml/ProgramProject/evaluation-predict/tmp/train/car_source_match.csv', index=False)

瓜子车源数据量: 1172847
瓜子车源数据匹配量: 902783 匹配率: 0.7697363765265205
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 902783 entries, 0 to 902782
Data columns (total 19 columns):
id                 902783 non-null int64
car_application    902783 non-null object
city               902783 non-null object
created_on         902783 non-null object
domain             902783 non-null object
mile               902783 non-null float64
month              902783 non-null int64
price              902783 non-null float64
title              902783 non-null object
transfer_owner     902783 non-null int64
year               902783 non-null int64
brand_slug         902783 non-null float64
model_slug         902783 non-null float64
detail_slug        902783 non-null float64
online_year        902783 non-null float64
price_bn           902783 non-null float64
brand_name         902783 non-null object
model_name         902783 non-null object
detail_name        902783 non-null object
dtypes: float64(7), int64(4),

In [8]:
data = final.loc[(final['online_year'].isnull()), :]
data.to_csv('../tmp/train/missing.csv',index=False)